#### **Creation of Simple Pos Tagging** 

import libraries

In [ ]:
import pandas as pd
import conllu
import math

In [ ]:
#loading the training corpus

# Open the .conllu file
with open("train.conllu", "r", encoding="utf-8") as file:
    # Read the contents of the file
    data = file.read()

In [ ]:
#loading the test corpus
#open the .conllu file

with open("test.conllu","r",encoding="utf-8") as f:
    #read the content
    content = f.read()
with open("test.pos","w",encoding="utf-8") as file:
    sen = conllu.parse(content)
    for s in sen:
        for t in s:
            word=t["form"]
            ps = t["upostag"]
            file.write(word + '\t' + ps +'\n')

In [ ]:
# Parse the .conllu data
import conllu
sentences = conllu.parse(data)

In [ ]:
with open("train.pos", 'w', encoding='utf-8') as file:
    for sentence in sentences:
    # Iterate over each token in the sentence
        for token in sentence:
        # Access token properties such as word form, lemma, part of speech, etc.
            word_form = token["form"]
            pos = token["upostag"]
            file.write(word_form + '\t' + pos +'\n')

In [ ]:
with open(r".\train.conllu", "r", encoding="utf-8") as f:
    train_data = f.read()
with open(r".\dev.conllu", "r", encoding="utf-8") as f:
    test_data = f.read()
train = {}
sentences = conllu.parse(train_data)

for sentence in sentences:
    for token in sentence:
        train[token['form']] = token['upos']
test = {}
sentences = conllu.parse(test_data)

for sentence in sentences:
    for token in sentence:
        test[token['form']] = token['upos']
vocab_l=list()
sentences = conllu.parse(train_data)

for sentence in sentences:
    for token in sentence:
        vocab_l.append(token['form'])
vocab_l=vocab_l+list(test.keys())

In [ ]:
len(vocab_l)

373634

START

In [ ]:
#loding train data
with open("train.pos", 'r',encoding='utf-8') as f:
    training_corpus = f.readlines()

print(f"A few items of the training corpus list")
print(training_corpus[0:5])

A few items of the training corpus list
['Les\tDET\n', 'commotions\tNFP\n', 'cérébrales\tADJFP\n', 'sont\tAUX\n', 'devenu\tVPPMS\n']


In [ ]:
# vocab: dictionary that has the index of the corresponding words
vocab = {} 

# Get the index of the corresponding words. 
for i, word in enumerate(sorted(set(vocab_l))): 
    vocab[word] = i       
    
print("Vocabulary dictionary, key is the word, value is a unique integer")
cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt > 20:
        break

Vocabulary dictionary, key is the word, value is a unique integer
!:0
":1
#:2
$:3
%:4
&:5
':6
'':7
'06:8
'07:9
'900:10
'upa'upa:11
(:12
):13
*:14
+:15
+0,5:16
+0,6:17
+1:18
+1,8:19
+14:20


In [ ]:
len(vocab)

44929

In [ ]:
# load in the test corpus
with open("test.pos", 'r') as f:
    y = f.readlines()

print("A sample of the test corpus")
print(y[0:10])

A sample of the test corpus
['Je\tPPER1S\n', 'sens\tVERB\n', "qu'\tCOSUB\n", 'entre\tPREP\n', 'Ã§a\tPDEMMS\n', 'et\tCOCO\n', 'les\tDET\n', 'films\tNMP\n', 'de\tPREP\n', 'mÃ©decins\tNMP\n']


In [ ]:
# testing words
with open("test.pos","r") as f:
    lines=f.readlines()

    first_column_words = []

# Parcourir chaque ligne et extraire le mot de la première colonne
for line in lines:
    # Diviser la ligne en colonnes en utilisant '\t' comme séparateur
    columns = line.split('\t')

    # Vérifier si le nombre de colonnes est suffisant
    if len(columns) >= 1:
        # Ajouter le mot de la première colonne à la liste
        first_column_words.append(columns[0])

# Ouvrir le fichier de sortie en écriture
with open('testing.words', 'w') as output_file:
    # Écrire les mots de la première colonne dans le fichier de sortie
    for w in first_column_words:
        output_file.write(w + '\n')
        


In [ ]:

def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep
def assign_unk(word):
    return "<UNK>"

def get_word_tag(line, vocabulary):
    if not line.strip():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        parts = line.split("\t", 1)
        if len(parts) != 2:
            return None, None
        word, tag = parts
        if word not in vocabulary: 
            word = assign_unk(word)
        return word, tag.strip()

In [ ]:
#corpus without tags, preprocessed
_, prep = preprocess(vocab, "testing.words")     

print('The length of the preprocessed test corpus: ', len(prep))
print('This is a sample of the test_corpus: ')
len(prep)

The length of the preprocessed test corpus:  10298
This is a sample of the test_corpus: 


10298

**TRAINING**

In [ ]:
def create_dictionaries(training_corpus, vocab):
    """
    Input: 
        training_corpus: a corpus where each line has a word followed by its tag.
        vocab: a dictionary where keys are words in vocabulary and value is an index
    Output: 
        emission_counts: a dictionary where the keys are (tag, word) and the values are the counts
        transition_counts: a dictionary where the keys are (prev_tag, tag) and the values are the counts
        tag_counts: a dictionary where the keys are the tags and the values are the counts
    """

    from collections import defaultdict
    # initialize the dictionaries using defaultdict
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '--s--' 
    
    # use 'i' to track the line number in the corpus
    i = 0 
    
    # Each item in the training corpus contains a word and its POS tag
    # Go through each word and its tag in the training corpus
    for word_tag in training_corpus: 
        
        # Increment the word_tag count
        i += 1
        
        # Every 50,000 words, print the word count
        if i % 50000 == 0:
            print(f"word count = {i}")
            
        ### START CODE HERE (Replace instances of 'None' with your code) ###
        # get the word and tag using the get_word_tag helper function (imported from utils_pos.py)
        word, tag = get_word_tag(word_tag,vocab) 
        
        # Increment the transition count for the previous word and tag
        transition_counts[(prev_tag, tag)] += 1
        
        # Increment the emission count for the tag and word
        emission_counts[(tag, word)] += 1

        # Increment the tag count
        tag_counts[tag] += 1

        # Set the previous tag to this tag (for the next iteration of the loop)
        prev_tag = tag
        
        ### END CODE HERE ###
        
    return emission_counts, transition_counts, tag_counts

In [ ]:
emission_counts, transition_counts, tag_counts = create_dictionaries(training_corpus, vocab)

word count = 50000
word count = 100000
word count = 150000
word count = 200000
word count = 250000
word count = 300000
word count = 350000


In [1]:
# emission_counts

In [2]:
# tag_counts

In [3]:
# transition_counts

In [ ]:
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 66
View these POS tags (states)
['ADJ', 'ADJFP', 'ADJFS', 'ADJMP', 'ADJMS', 'ADV', 'AUX', 'CHIF', 'COCO', 'COSUB', 'DET', 'DETFS', 'DETMS', 'DINTFS', 'DINTMS', 'INTJ', 'MOTINC', 'NFP', 'NFS', 'NMP', 'NMS', 'NOUN', 'NUM', 'PART', 'PDEMFP', 'PDEMFS', 'PDEMMP', 'PDEMMS', 'PINDFP', 'PINDFS', 'PINDMP', 'PINDMS', 'PINTFS', 'PPER1S', 'PPER2S', 'PPER3FP', 'PPER3FS', 'PPER3MP', 'PPER3MS', 'PPOBJFP', 'PPOBJFS', 'PPOBJMP', 'PPOBJMS', 'PREF', 'PREFP', 'PREFS', 'PREL', 'PRELFP', 'PRELFS', 'PRELMP', 'PRELMS', 'PREP', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'VPPFP', 'VPPFS', 'VPPMP', 'VPPMS', 'VPPRE', 'X', 'XFAMIL', 'YPFOR', '_']


In [ ]:
print("transition examples: ")
for ex in list(transition_counts.items())[:3]:
    print(ex)
print()

print("emission examples: ")
for ex in list(emission_counts.items())[200:203]:
    print (ex)
print()

print("ambiguous word example: ")
for tup,cnt in emission_counts.items():
    if tup[1] == 'comparable':
        print (tup, cnt) 

transition examples: 
(('--s--', 'DET'), 1)
(('DET', 'NFP'), 4162)
(('NFP', 'ADJFP'), 1452)

emission examples: 
(('ADJMP', 'autres'), 135)
(('ADJMP', 'petits'), 29)
(('NMP', 'fruits'), 9)

ambiguous word example: 
('ADJMS', 'comparable') 1
('ADJFS', 'comparable') 4


**TESTING**

In [ ]:
def predict_pos(prep, y, emission_counts, vocab, states):
    '''
    Input: 
        prep: a preprocessed version of 'y'. A list with the 'word' component of the tuples.
        y: a corpus composed of a list of tuples where each tuple consists of (word, POS)
        emission_counts: a dictionary where the keys are (tag,word) tuples and the value is the count
        vocab: a dictionary where keys are words in vocabulary and value is an index
        states: a sorted list of all possible tags for this assignment
    Output: 
        accuracy: Number of times you classified a word correctly
    '''
    
    # Initialize the number of correct predictions to zero
    num_correct = 0
    
    # Get the (tag, word) tuples, stored as a set
    all_words = set(emission_counts.keys())
    
    # Get the number of (word, POS) tuples in the corpus 'y'
    total = len(y)
    for word, y_tup in zip(prep, y): 

        # Split the (word, POS) string into a list of two items
        y_tup_l = y_tup.split()
        
        # Verify that y_tup contain both word and POS
        if len(y_tup_l) == 2:
            
            # Set the true POS label for this word
            true_label = y_tup_l[1]

        else:
            # If the y_tup didn't contain word and POS, go to next word
            continue
    
        count_final = 0
        pos_final = ''
        
        # If the word is in the vocabulary...
        if word in vocab:
            for pos in states:
                        
                # define the key as the tuple containing the POS and word
                key = (pos,word)

                # check if the (pos, word) key exists in the emission_counts dictionary
                if key in emission_counts: # complete this line

                # get the emission count of the (pos,word) tuple 
                    count = emission_counts[key]

                    # keep track of the POS with the largest count
                    if count>count_final: # complete this line

                        # update the final count (largest count)
                        count_final = count

                        # update the final POS
                        pos_final = pos

            # If the final POS (with the largest count) matches the true POS:
            if pos_final == true_label: # complete this line
                
                # Update the number of correct predictions
                num_correct += 1
            
    ### END CODE HERE ###
    accuracy = num_correct / total
    
    return accuracy

In [ ]:
accuracy_predict_pos = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Accuracy of prediction using predict_pos is {accuracy_predict_pos:.4f}")

Accuracy of prediction using predict_pos is 0.7606
